# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding. 

## Use the previously created CSV file called <font color=red>event_datafile_new.csv</font>.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Create a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

except Exception as e:
    print (e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## The data model model needs to be set up to answer the following three questions:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

For table song_in_session artist, song_title and song_length columns are added because this is the information that the query will request. The columns item_in_session and session_id columns are used as Composite Partition Keys in order to enable using the WHERE clause to perform the required query. 

In [8]:
query_create_table_song_in_session = "CREATE TABLE IF NOT EXISTS song_in_session (item_in_session int, session_id int, artist text, song_title text, song_length float, PRIMARY KEY(item_in_session, session_id))"

try: 
    session.execute(query_create_table_song_in_session)
except Exception as e:
    print (e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_in_session (item_in_session, session_id, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query1 = "SELECT artist, song_title, song_length FROM song_in_session WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query1)
except Exception as e:
    print (e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For table artist_in_session user_id, session_id,item_in_session, artist, song and user columns are added because this is the information that the query will request. The columns user_id and session_id are used as Composite Partition Keys to enable using the WHERE clause to refine a query based on these columns. The column item_in_session is used as a Clustering Key in order to be able to perform a query that orders the data by the item_in_session column. 

In [11]:
query_create_table_artist_in_session = "CREATE TABLE IF NOT EXISTS artist_in_session (user_id int, session_id int, item_in_session int, artist text, song text, user text, PRIMARY KEY((user_id, session_id), item_in_session))"

try: 
    session.execute(query_create_table_artist_in_session)
except Exception as e:
    print (e)      

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_in_session (user_id, session_id, item_in_session, artist, song, user)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9] ,str(line[1] + " " + line[4])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query2 = "SELECT artist, song, user FROM artist_in_session WHERE user_id=10 AND session_id=182"
try: 
    rows = session.execute(query2)
except Exception as e:
    print (e)  
    
for row in rows:
    print (row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

For table user_song columns song, user_id, first_name and last_name are added because this is the information that the query will request. The columns song and user_id are used as Composite Partition Keys to enable using the WHERE clause to refine a query based on these columns.

In [14]:
query_create_table_user_song = "CREATE TABLE IF NOT EXISTS user_song (song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"

try: 
    session.execute(query_create_table_user_song)
except Exception as e:
    print (e)       

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
query2 = "SELECT first_name, last_name FROM user_song WHERE song='All Hands Against His Own'"
try: 
    rows = session.execute(query2)
except Exception as e:
    print (e)  
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
tables = ["user_song", "artist_in_session", "song_in_session"]
for table in tables: 
    query = "DROP TABLE IF EXISTS " + table 
    session.execute(query)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()